In [17]:
import tweepy
from textblob import TextBlob

# Data fetcher APIs
import GetOldTweets3 as got
import snscrape.modules.twitter as sntwitter

import pandas as pd
import dask.dataframe as dd
import dask
import numpy as np
import matplotlib.pyplot as plt

import re

### Attempt Twitter API

In [31]:
twitter_credentials = pd.read_csv('twitter_keys.csv')
consumer_key = twitter_credentials['API_KEY'][0]
consumer_secret = twitter_credentials['API_KEY_SECRET'][0]
bearer_token = twitter_credentials['BEARER_TOKEN'][0]
access_token = twitter_credentials['ACCESS_TOKEN'][0]
access_token_secret = twitter_credentials['ACCESS_TOKEN_SECRET'][0]

In [32]:
auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth)

In [41]:
# Filter out retweets
search_term = '#bitcoin -filter:retweets'

tweets = tweepy.Cursor(
    api.search_tweets,
    q=search_term,
    lang='en',
#     since='2018-01-01',
    tweet_mode='extended'
).items(2000)

In [42]:
all_tweets = [tweet.full_text for tweet in tweets]

In [44]:
len(all_tweets)

2000

**Cannot use Twitter API as the tweets are limited to only the past 7 days**

### Attempt GOT package

In [72]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch(search_term)\
                                           .setSince("2019-03-29")\

tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: HTTP Error 403: Forbidden
Try to open in browser: https://twitter.com/search?q=%23bitcoin%20-filter%3Aretweets%20since%3A2019-03-29&src=typd
Traceback (most recent call last):
  File "c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\GetOldTweets3\manager\TweetManager.py", line 343, in getJsonResponse
    response = opener.open(url)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "c:\users\acer\appdata\local\programs\python\python37\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "c:\users\acer\appdata\l

TypeError: object of type 'NoneType' has no len()

Also rendered useless due to the Twitter restrictions

### Attempt Twitter scraper

In [76]:
tweets_list = []

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(
    '#bitcoin -filter:retweets since:2019-03-29'
).get_items()):
    tweets_list.append([tweet.date, tweet.content, tweet.username])


ERROR:snscrape.base:Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%23bitcoin+-filter%3Aretweets+since%3A2019-03-29&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaEwNHhm8XmhSwWgICqjYyb7o0sEnEVk_-SARWAiXoYBFVTRVI1ARXG-wEVAAA%3D&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment: non-200 status code
CRITICAL:snscrape.base:4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_composer_source=true&include_ext_alt_text=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweets=true&q=%23bitcoin+-filter%3Aretweets+since%3A2019-03-29&tweet_search_mode=live&count=100&query_source=spelling_expansion_revert_click&cursor=scroll%3AthGAVUV0VFVBaEwNHhm8XmhSwWgICqjYyb7o0sEnEVk_-SARWAiXoYBFVTRVI1ARXG-wEVAAA%3D&pc=1&spelling_corrections=1&ext=mediaStats%252CcameraMoment failed, giving up.

In [80]:
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet', 'Username'])

In [83]:
tweets_df.head()

,Datetime,Tweet,Username
0,2022-11-06 08:11:34+00:00,I see #Bitcoin slowly becoming a substitute c...,jordiii24797
1,2022-11-06 08:11:30+00:00,#Bitcoin https://t.co/y5QYpqXEDQ,time2trade
2,2022-11-06 08:11:29+00:00,Preguntas y respuestas sobre #Bitcoin\nDía 2:\...,Leo_Va_Dinci
3,2022-11-06 08:11:29+00:00,#Bitcoin Going Parabolic! 🔥👇 https://t.co/mZNu...,nahn_nuhibuk_ya
4,2022-11-06 08:11:27+00:00,It's weekend. Decided to zoom out on #Bitcoin'...,prxsxd_0


In [82]:
tweets_df.tail()

,Datetime,Tweet,Username
220412,2022-10-31 01:56:53+00:00,Top 5 #cryptocurrency #price jumps in last min...,_Crypto_Frog
220413,2022-10-31 01:56:49+00:00,Neither Brazil will ban #bitcoin,johnpham88835
220414,2022-10-31 01:56:49+00:00,Bitcoin is crap\n\n😤😡😤😤😠\n$BTC $XBT #Bitcoin $...,techDespiser
220415,2022-10-31 01:56:42+00:00,BRAND NEW UNIT Antminer S19 Pro 110t/h BTC Bit...,theincomeblog
220416,2022-10-31 01:56:41+00:00,#Bitcoin Last Price $20542 #BTC 🚀\nWeekly Indi...,LivePriceCrypto


In [84]:
tweets_df.to_csv('../../data/BTC_Tweets_snscrape.csv')

Also rendered useless: only past week's tweets were obtained

### Kaggle dataset

[data small](https://www.kaggle.com/datasets/kaushiksuresh147/bitcoin-tweets)
[data large](https://www.kaggle.com/datasets/alaix14/bitcoin-tweets-20160101-to-20190329)

*Attempt pandas*

In [2]:
df_small = pd.read_csv('../../data/Bitcoin_tweets.csv')

KeyboardInterrupt: 

Pandas takes so long and errors out

*Attempt dask*

In [14]:
df_small = dd.read_csv(
    '../../data/Bitcoin_tweets.csv',
    usecols=['user_name', 'user_description', 'date', 'text', 'hashtags']
)

In [15]:
df_small.head()

,user_name,user_description,date,text,hashtags
0,DeSota Wilson,"Biz Consultant, real estate, fintech, startups...",2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin']
1,CryptoND,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']"
2,Tdlmatias,"IM Academy : The best #forex, #SelfEducation, ...",2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN
3,Crypto is the future,I will post a lot of buying signals for BTC tr...,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']"
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC']


Was able to read the small dataset, attempt large

In [55]:
df_large = dd.read_csv(
    '../../data/Bitcoin_tweets_large.csv',
    delimiter=';',
    usecols=['user', 'fullname', 'timestamp', 'text'],
    engine='python',
    on_bad_lines='skip'
)

In [56]:
df_large.head()

,user,fullname,timestamp,text
0,KamdemAbdiel,Abdiel kamdem,2019-05-27 11:49:14+00,È appena uscito un nuovo video! LES CRYPTOMONN...
1,bitcointe,Bitcointe,2019-05-27 11:49:18+00,Cardano: Digitize Currencies; EOS https://t.co...
2,3eyedbran,Bran - 3 Eyed Raven,2019-05-27 11:49:06+00,Another Test tweet that wasn't caught in the s...
3,DetroitCrypto,J. Scardina,2019-05-27 11:49:22+00,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,mmursaleen72,Muhammad Mursaleen,2019-05-27 11:49:23+00,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


In [61]:
df_large['timestamp'] = dd.to_datetime(df_large['timestamp'])

In [64]:
df_large.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, user to text
dtypes: datetime64[ns](1), object(3)

In [71]:
# Check if any null timestamps
df_large['timestamp'].isnull().sum().compute()

823

In [77]:
df_large.size.compute()

67562152

In [80]:
# Remove empty timestamp values to facilitate sorting
df_large_cleaned = df_large.dropna(subset=['timestamp'])

In [84]:
df_large_cleaned.sort_values('timestamp', inplace=True).compute()

,user,fullname,timestamp,text
114517,chrispychong,chrispy,2007-04-19 07:14:38+00:00,is happily mugging at BTC where she will hook ...
21,halfin,halfin,2009-01-11 03:33:52+00:00,Running bitcoin
45666,halfin,halfin,2009-01-21 17:29:40+00:00,Looking at ways to add more anonymity to bitcoin
91597,halfin,halfin,2009-01-27 20:14:10+00:00,Thinking about how to reduce CO2 emissions fro...
18020,fafcffacfff,GoldLover,2009-01-29 13:37:53+00:00,From: Satoshi Nakamoto - 2009-01-11 22:32 Bitc...
...,...,...,...,...
139141,theautomatski,Automatski,2019-11-23 15:45:53+00:00,The Worlds First Millennium Firm https://t.co/...
139143,digital_mine_,digital mine ⚡🤖👨‍💻🇭🇰🇭🇰🇭🇰,2019-11-23 15:45:55+00:00,Daily profit for HODLING BTC since 2013 Data t...
139145,Vizique,Vizique,2019-11-23 15:45:55+00:00,Bitcoin Suisse Certificates :) https://t.co/nd...
139146,torusJKL,Gal Buki ($torusJKL),2019-11-23 15:45:56+00:00,Register now for the early access of the Codug...


In [83]:
df_large_cleaned.tail()

,user,fullname,timestamp,text
139144,JacobCanfield,Jacob Canfield,2019-11-23 15:28:50+00:00,Happy #FibonacciDay \n\nA while back I created...
139145,Vizique,Vizique,2019-11-23 15:45:55+00:00,Bitcoin Suisse Certificates :) https://t.co/nd...
139146,torusJKL,Gal Buki ($torusJKL),2019-11-23 15:45:56+00:00,Register now for the early access of the Codug...
139147,Adekunl95628158,Adekunle Daniel,2019-11-23 15:45:57+00:00,@btc \n@btc \nDo you know that BTC Baskets isn...
139148,HaraldoXRP,Harry,2019-11-23 15:45:06+00:00,$BTC - an update on the longer term view for B...


In [98]:
# Remove unneeded time section
df_large_cleaned['timestamp'] = df_2_cleaned['timestamp'].dt.strftime('%m/%d/%Y')

In [134]:
df_large_cleaned.head()

,user,fullname,timestamp,text
0,KamdemAbdiel,Abdiel kamdem,05/27/2019,È appena uscito un nuovo video! LES CRYPTOMONN...
1,bitcointe,Bitcointe,05/27/2019,Cardano: Digitize Currencies; EOS https://t.co...
2,3eyedbran,Bran - 3 Eyed Raven,05/27/2019,Another Test tweet that wasn't caught in the s...
3,DetroitCrypto,J. Scardina,05/27/2019,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,mmursaleen72,Muhammad Mursaleen,05/27/2019,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


In [149]:
df_large_cleaned.tail()

,user,fullname,timestamp,text
139144,JacobCanfield,Jacob Canfield,11/23/2019,Happy #FibonacciDay \n\nA while back I created...
139145,Vizique,Vizique,11/23/2019,Bitcoin Suisse Certificates :) https://t.co/nd...
139146,torusJKL,Gal Buki ($torusJKL),11/23/2019,Register now for the early access of the Codug...
139147,Adekunl95628158,Adekunle Daniel,11/23/2019,@btc \n@btc \nDo you know that BTC Baskets isn...
139148,HaraldoXRP,Harry,11/23/2019,$BTC - an update on the longer term view for B...


In [150]:
df_large_cleaned.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, user to text
dtypes: object(4)

In [136]:
# Filter old data
df_large_filtered = df_large_cleaned[df_large_cleaned['timestamp'] > '01/01/2014']

In [151]:
df_large_filtered.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, user to text
dtypes: object(4)

In [137]:
df_large_filtered.size.compute()

67554192

In [ ]:
# Set date as index to repartition
df_large_filtered_indexed = df_large_filtered.set_index('timestamp')

In [144]:
df_large_filtered_indexed.head()

,user,fullname,text
timestamp,,,
01/01/2015,BTCticker,bitcointicker.co,One Bitcoin now worth $315.63@bitstamp. High $...
01/01/2015,ProjectCoin,Project Coin,LIVE: Profit = $712.36 (27.66 %). BUY B8.16 @ ...
01/01/2015,airdroplite,Airdrop Lite,BTCTurk 762.55 TL Koinim 770 TL CampBx 330.00 ...
01/01/2015,BTCticker,bitcointicker.co,One Bitcoin now worth $314.75@bitstamp. High $...
01/01/2015,BitcoinSpreads,Bitcoin Spreads,1 #BTC (#Bitcoin) quotes:\n$314.62/$315.00 #Bi...


In [145]:
df_large_filtered_indexed.tail()

,user,fullname,text
timestamp,,,
12/31/2018,Uberbills,Uberbills.com,"BTC: $3751.00, S: $Looking Fo, G: $ Spot Pric ..."
12/31/2018,bt_exchange,Bitcoin Exchange BR,1 BTC = 15228.57976000 BRL em 31/12/2018 ás 12...
12/31/2018,BitcoinCash_ES,BitcoinCash_es,Cotización del Bitcoin Cash: 136 80.€ | -0.15%...
12/31/2018,asens_inc,ASENS inc.,"12/31 23:00現在\n\n#Bitcoin : 410,395円↓\n#NEM #X..."
12/31/2018,short_vs_long,SHORT vs LONG(Beta),24H\n2019/01/01 02:00 (2018/12/31 01:59)\nLONG...


In [146]:
# Dataframe is extremely large - split into chunks
df_large_repartitioned = df_large_filtered_indexed.repartition(partition_size='100MB')

AttributeError: 'DataFrame' object has no attribute 'repartition'

In [59]:
df_2_repartitioned.to_csv('../../data/dataframes/data-*.csv')

KeyboardInterrupt: 

Process limited by available memory -> Moving to Kaggle as it has 30GB RAM

* `break_file_days.ipynb`